<span style="color: green; font-size: 40px; font-weight: bold;">Projeto 3 (Parte 2)</span>

<br>

<span style="color: green; font-size: 40px; font-weight: bold;">Sistema de Recomendação em Tempo Real</span>

<br> <br>

# Importante

> É necessário que a célula da etapa `Criação do Kafka Producer e Envio de Dados para o Kafka` da **Parte 1** do projeto esteja rodando.

<br><br><br>

# Importando Pacotes

<br>

In [ ]:
# Imports
import os
import time
import random
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler